In [1]:
import re
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text) #keep words/numbers/whitespace
    return text.split()

#test preprocessing
print(preprocess("Hello! How's the weather?"))

['hello', 'hows', 'the', 'weather']


In [ ]:
import re
from collections import defaultdict
from preprocess import preprocess

class NGramModel:
    def __init__(self, n=3):
        self.n = n
        self.ngrams = defaultdict(lambda: defaultdict(int))
        self.vocab = set()

    def train(self, corpus):
        #collect n-grams and vocabulary
        print(f"Training on {len(corpus)} sentences...")
        for sentence in corpus:
            tokens = preprocess(sentence)
            if not tokens:
                continue
            self.vocab.update(tokens)
            print(f"Processing sentence: {tokens}")  # Debug

            for i in range(len(tokens)-self.n+1):
                context = tuple(tokens[i:i+self.n-1])
                next_word = tokens[i+self.n-1]
                self.ngrams[context][next_word]+=1
                print(f"Added n-gram: {context} -> {next_word}")  # Debug

        #add laplace smoothing
        self._apply_smoothing()
        print(f"Vocabulary size: {len(self.vocab)}")
        print(f"Total contexts: {len(self.ngrams)}")

    def _apply_smoothing(self, alpha=0.1):
        vocab_size = len(self.vocab)
        for context in self.ngrams:
            total = sum(self.ngrams[context].values())+alpha*vocab_size
            for word in self.vocab:
                self.ngrams[context][word] = (self.ngrams[context].get(word, 0) + alpha)/total

    #enhanced prediction with backoff
    def predict(self, text, top_k=3):
        tokens = preprocess(text)
        print(f"Input tokens: {tokens}")  # Debug

        # Handle context window
        context = tuple(tokens[-(self.n-1):]) if len(tokens) >= self.n-1 else tuple(tokens)
        print(f"Initial context: {context}")  # Debug


        #backoff mechanism
        while len(context)>0:
            if context in self.ngrams:
                suggestions = sorted(self.ngrams[context].items(),key=lambda x: -x[1])[:top_k]
                return [word for word, prob in suggestions]
            context = context[1:] if len(context) > 1 else tuple()
        print(f"Backing off to: {context}")  # Debug
        #remove oldest word for backoff
            
        
        #final fallback to unigrams (most common words)
# Final fallback to most common words
        common_words = sorted(self.vocab, 
                            key=lambda x: sum(m[x] for m in self.ngrams.values()),
                            reverse=True)[:top_k]
        return common_words

['hello', 'hows', 'the', 'weather']


In [11]:
from ngram_model import NGramModel

with open('textfile.txt', 'r', encoding='utf-8') as file:
    corpus_text = file.read()

#initialize and train model
model = NGramModel(n=3)
model.train(corpus=corpus_text)

#test predictions
print(model.predict("I enjoy"))

print(model.predict("science"))

print(model.predict("unknown text"))

[]
[]
[]
